<a href="https://colab.research.google.com/github/DmitrijLeontev/Dmitrij_Leontev/blob/main/DzDatascience/dz13_%D0%9C%D0%BE%D1%8F_%D0%BF%D0%B5%D1%80%D0%B2%D0%B0%D1%8F_%D1%81%D0%B2%D0%B5%D1%80%D1%82%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D1%81%D0%B5%D1%82%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Создание простой модели сверточной нейронной сети- Моя первая модель сверточной нейронной сети!!!

Подключите основу – класс создания последовательной модели Sequential:

In [ ]:
from tensorflow.keras.models import Sequential

С помощью него создайте экземпляр вашей модели:

In [ ]:
model = Sequential()

Это и есть ваша модель! Сейчас она больше похожа на пустую коробку. Чтобы она что-то делала, нужно поместить в нее какой-нибудь механизм. Это не механизм в обычном смысле слова, потому что вы будете оперировать не предметами, а информацией – главным ресурсом XXI века. Механизм будет принимать на вход и выдавать на выход какие-то данные.

Так из чего же вы можете создать механизм? Для начала определитесь, сколько информации вы будете давать нейросети на вход. Один экземпляр такой информации называется объектом. Не углубляйтесь пока, какими они бывают и как устроены. Сейчас достаточно знать, что объекты всегда состоят из чисел.

Например, вы решили, что ваши объекты - изображения. Для подачи в нейросеть их надо оцифровать.

У изображений есть высота img_height, ширина img_width и количество цветовых каналов channels.

Их называют входной формой (или формой входных данных) и записывают как:

input_shape=(img_height, img_width, channels)

Важно: все изображения, которые вы подаете на вход нейронной сети, должны иметь общие высоту, ширину и количество каналов. Ниже вы узнаете, как это сделать.

Добавьте в модель первый слой при помощи .add():

In [ ]:
from tensorflow.keras.layers import Conv2D

# Первый сверточный слой
model.add(Conv2D(8, (3, 3), padding='same', activation='relu', input_shape=(28, 56, 3)))

Расшифруем написанное выше: первый сверточный слой принимает на вход цветное изображение (3 канала) размерами 28 на 56 пикселей. То есть форма входящего массива - (28, 56, 3).

Внутри слоя к нему применяется свертка 8-ю фильтрами (3, 3) с шагом смещения (1, 1), а затем функция активации relu.

Какой формы получится выходной массив?

Обратите внимание, что padding ='same', stride=(1,1) по умолчанию; вычислим pad = (size - 1) / 2 = (3-1) /2 = 1.

Используем формулу:

output_h = (input_h + 2 * pad - size) // stride + 1 = (28 + 2 * 1 - 3) // 1 + 1 = 27 + 1 = 28
output_w = (input_w + 2 * pad - size) // stride + 1 = (56 + 2 * 1 - 3) // 1 + 1 = 55 + 1 = 56
Как видите, при стандартном stride и padding ='same' длина и ширина входного и выходного массивов сверточного слоя равны. Но отличие все-таки будет - это глубина!

На вход пришло 3 канала, а сверточный слой имеет 8 ядер свертки, каждое из которых выдает свою карту признаков, обработав любое количество каналов. Значит, глубина на выходе будет 8 вместо 3. Полная форма данных на выходе получится (28, 56, 8).

Проверьте это методом модели .summary():

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 56, 8)         224       
                                                                 
Total params: 224 (896.00 Byte)
Trainable params: 224 (896.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Отлично!

Посмотрим, что будет, если добавить следующий сверточный слой c 5 фильтрами с ядром (3, 2), шагом смещения (2, 3) и padding ='valid':

In [ ]:
# Второй сверточный слой
model.add(Conv2D(5, (3, 2), strides = (2,3), padding='valid', activation='relu'))

Если padding='valid', то по правилам pad = 0. Подставим значения:

    output_h = (input_h + 2 * pad - size) // stride + 1 = (28 + 2 * 0 - 3) // 2 + 1 = 25 // 2 + 1 = 12 + 1 = 13
    output_w = (input_w + 2 * pad - size) // stride + 1 = (56 + 2 * 0 - 2) // 3 + 1 = 54 // 3 + 1 = 18 + 1 = 19
У вас 5 фильтров, значит форма данных на выходе получится (13, 19, 5).

Проверьте:

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 56, 8)         224       
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 19, 5)         245       
                                                                 
Total params: 469 (1.83 KB)
Trainable params: 469 (1.83 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Примените слой MaxPooling2D:

In [ ]:
from tensorflow.keras.layers import MaxPooling2D

# Слой подвыборки
model.add(MaxPooling2D(pool_size=(3, 3)))

MaxPooling2D изменит форму данных следующим образом (учитывая, что stride=pool_size):

 output_h = (input - pool_size) // strides + 1 = (13 - 3) // 3 + 1 = 4
 output_w = (input - pool_size) // strides + 1 = (19 - 3) // 3 + 1 = 6
Глубина в MaxPooling2D не меняется, выходная форма данных (4, 6, 5).

Проверьте:

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 56, 8)         224       
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 19, 5)         245       
                                                                 
 max_pooling2d (MaxPooling2  (None, 4, 6, 5)           0         
 D)                                                              
                                                                 
Total params: 469 (1.83 KB)
Trainable params: 469 (1.83 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Далее примените слой Flatten, который вытягивает входящий тензор в одномерный вектор. На входе слоя ожидается тензор (4, 6, 5), а на выходе будет вектор (4 * 6 * 5) = (120)

In [ ]:
from tensorflow.keras.layers import Flatten

# Слой преобразования многомерных данных в одномерные
model.add(Flatten())

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 56, 8)         224       
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 19, 5)         245       
                                                                 
 max_pooling2d (MaxPooling2  (None, 4, 6, 5)           0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 120)               0         
                                                                 
Total params: 469 (1.83 KB)
Trainable params: 469 (1.83 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Нужно обратить внимание, что размерность первых двух элементов тензора перед Flatten слоем (в этом случае, 4 х 6) не должна быть очень большая - можно добавлять слои Conv2D и MaxPooling2D пока эта размерность не станет равна 1 х 1. Например, если бы мы не добавили слой MaxPooling2D, эта размерность была бы равна 13 х 19, что бы было уже однозначно много.

Если размерность, которая мы подаем в Flatten() слой слишком большая, следующий слой не сможет извлечь достаточно значемые признаки для правильной классификации, потому что на вход слоя приходят слишком много данных.

Далее мы создадим последний, выходной слой Dense. Он получит на вход одномерный вектор (120), а на выходе выдаст одномерный вектор (3). Активационная функция softmax выдаст вероятности принадлежности входных данных к каждому из трех классов.

Проверьте:

In [ ]:
from tensorflow.keras.layers import Dense

model.add(Dense(3, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 56, 8)         224       
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 19, 5)         245       
                                                                 
 max_pooling2d (MaxPooling2  (None, 4, 6, 5)           0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 120)               0         
                                                                 
 dense (Dense)               (None, 3)                 363       
                                                                 
Total params: 832 (3.25 KB)
Trainable params: 832 (3.25 KB)
Non-trainable params: 0 (0.00 Byte)
__________________________

Вот вы и построили простую сверточную сеть. Теперь у вас есть небольшой опыт, который пригодится для построения более сложной сети, чтобы решить задачу классификации.

https://colab.research.google.com/drive/1gvcOKclIUhie9WfKRr27fB9bJ1quDNVe?usp=sharing